In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ct = pd.read_csv("../csvs/htaindex_data_tracts_06.csv")
crimes = pd.read_csv("../csvs/crimes.csv")
schools = pd.read_csv("../csvs/new_district.csv")
ct_info = gpd.read_file("../tl_2017_06_tract/tl_2017_06_tract.shp")

In [3]:
ct.tract = ct.tract.apply(lambda k: k[1:-1])
ct.head()

,tract,blkgrps,population,households,land_acres,ht_ami,ht_80ami,ht_nmi,h_ami,h_80ami,...,emp_gravity,emp_ndx,block_size,intersection_density,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu
0,06001400100,1.0,2952.0,1286.0,1703.63,76.0,92.0,112.0,57.0,71.0,...,44078.0,92.0,21.0,70.0,1334,3848.0,4001.0,2944.0,86.0,14.0
1,06001400200,2.0,1984.0,832.0,145.16,54.0,66.0,80.0,38.0,48.0,...,69965.0,93.0,4.0,383.0,542,2605.0,3097.0,1775.0,64.0,36.0
2,06001400300,4.0,5377.0,2489.0,273.13,44.0,53.0,64.0,29.0,36.0,...,81494.0,93.0,3.0,478.0,529,1973.0,3097.0,1077.0,44.0,56.0
3,06001400400,3.0,4105.0,1801.0,176.61,45.0,54.0,65.0,30.0,37.0,...,76587.0,93.0,3.0,408.0,457,2036.0,2302.0,1537.0,38.0,62.0
4,06001400500,3.0,3651.0,1624.0,145.87,41.0,50.0,61.0,27.0,34.0,...,81371.0,94.0,3.0,405.0,487,1836.0,2686.0,1354.0,36.0,64.0


In [6]:
ct.columns

Index(['tract', 'blkgrps', 'population', 'households', 'land_acres', 'ht_ami',
       'ht_80ami', 'ht_nmi', 'h_ami', 'h_80ami', 'h_nmi', 't_ami', 't_80ami',
       't_nmi', 'co2_per_hh_local', 'co2_per_acre_local', 'autos_per_hh_ami',
       'autos_per_hh_80ami', 'autos_per_hh_nmi', 'vmt_per_hh_ami',
       'vmt_per_hh_80ami', 'vmt_per_hh_nmi', 'pct_transit_commuters_ami',
       'pct_transit_commuters_80ami', 'pct_transit_commuters_nmi',
       't_cost_ami', 't_cost_80ami', 't_cost_nmi', 'auto_ownership_cost_ami',
       'auto_ownership_cost_80ami', 'auto_ownership_cost_nmi', 'vmt_cost_ami',
       'vmt_cost_80ami', 'vmt_cost_nmi', 'transit_cost_ami',
       'transit_cost_80ami', 'transit_cost_nmi', 'transit_trips_ami',
       'transit_trips_80ami', 'transit_trips_nmi', 'compact_ndx',
       'emp_ovrll_ndx', 'res_density', 'gross_hh_density', 'hh_gravity',
       'frac_sfd', 'emp_gravity', 'emp_ndx', 'block_size',
       'intersection_density', 'avg_block_perimeter_meters', 'h_cost',


In [7]:
ct_info.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,001,442700,06001442700,4427,Census Tract 4427,G5020,S,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3..."
1,06,001,442800,06001442800,4428,Census Tract 4428,G5020,S,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3..."
2,06,037,204920,06037204920,2049.20,Census Tract 2049.20,G5020,S,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3..."
3,06,037,205110,06037205110,2051.10,Census Tract 2051.10,G5020,S,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3..."
4,06,037,205120,06037205120,2051.20,Census Tract 2051.20,G5020,S,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3..."


In [8]:
ct_info.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [4]:
ct_info_to_join = ct_info[['COUNTYFP', 'GEOID', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']]

In [5]:
len(ct)

8036

In [6]:
len(ct_info)

8057

In [7]:
tracts = pd.merge(ct_info_to_join, ct, how='outer', left_on='GEOID', right_on='tract')
tracts.head()

,COUNTYFP,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract,blkgrps,population,...,emp_gravity,emp_ndx,block_size,intersection_density,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu
0,001,06001442700,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3...",06001442700,3.0,3275.0,...,36491.0,92.0,10.0,237.0,1145.0,2304.0,2581.0,1294.0,75.0,25.0
1,001,06001442800,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3...",06001442800,1.0,3056.0,...,38710.0,93.0,8.0,243.0,688.0,2205.0,2303.0,1958.0,71.0,29.0
2,037,06037204920,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3...",06037204920,2.0,2570.0,...,115862.0,95.0,5.0,222.0,714.0,1322.0,1605.0,1067.0,48.0,52.0
3,037,06037205110,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3...",06037205110,1.0,3826.0,...,143759.0,96.0,6.0,325.0,830.0,1140.0,NaN,1140.0,0.0,100.0
4,037,06037205120,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3...",06037205120,2.0,3288.0,...,131817.0,95.0,4.0,210.0,708.0,766.0,NaN,766.0,12.0,88.0


In [8]:
len(tracts)

8057

In [9]:
counties = gpd.read_file("../tl_2017_us_county/tl_2017_us_county.shp")
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,,,,A,1477641638,10701538,+41.9158651,-096.7885168,"POLYGON ((-97.019516 42.004097, -97.019519 42...."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,,,,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-123.436394 46.238197, -123.447592 4..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,,,,A,6016761648,29147345,+34.3592729,-104.3686961,"POLYGON ((-104.567387 33.997566, -104.567717 3..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,,A,2169252486,22867561,+40.7835474,-096.6886584,"POLYGON ((-96.91059899999999 40.958407, -96.91..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,,,,A,1489645186,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27366699999999 40.089399, -98.27..."


In [10]:
counties.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [11]:
ca_counties = counties[counties['STATEFP'] == '06']
ca_counties.drop("geometry", axis=1, inplace=True)
ca_counties.head()

/Users/kanamishra/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
8,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,,,,A,2468694586,23299110,+39.5769252,-120.5219926
325,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,,A,2499039655,76217822,+38.4500161,-121.3404408
329,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,,42200,,A,7084000622,2729814515,+34.5370572,-120.0399729
346,06,009,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,,,,A,2641829199,43797662,+38.1838996,-120.5614415
394,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,37100,,A,4773302741,946030569,+34.3587415,-119.1331432


In [12]:
ca_counties_to_merge = ca_counties[['COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD']]

In [13]:
len(tracts_with_county)

NameError: name 'tracts_with_county' is not defined

In [17]:
tracts_with_county = pd.merge(tracts, ca_counties_to_merge, how='left', on='COUNTYFP')
tracts_with_county.head()

,COUNTYFP,GEOID_x,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract,blkgrps,population,...,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu,COUNTYNS,GEOID_y,NAME,NAMELSAD
0,001,06001442700,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3...",06001442700,3.0,3275.0,...,1145.0,2304.0,2581.0,1294.0,75.0,25.0,01675839,06001,Alameda,Alameda County
1,001,06001442800,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3...",06001442800,1.0,3056.0,...,688.0,2205.0,2303.0,1958.0,71.0,29.0,01675839,06001,Alameda,Alameda County
2,037,06037204920,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3...",06037204920,2.0,2570.0,...,714.0,1322.0,1605.0,1067.0,48.0,52.0,00277283,06037,Los Angeles,Los Angeles County
3,037,06037205110,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3...",06037205110,1.0,3826.0,...,830.0,1140.0,NaN,1140.0,0.0,100.0,00277283,06037,Los Angeles,Los Angeles County
4,037,06037205120,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3...",06037205120,2.0,3288.0,...,708.0,766.0,NaN,766.0,12.0,88.0,00277283,06037,Los Angeles,Los Angeles County


In [18]:
schools[schools["Rank score (2017)"].isnull()]

,Unnamed: 0,index,Rank,District,Grades,# Elementary Schools,# Middle Schools,# High Schools,# Alt. Schools,City,County,Rank score (2017),Rank (2016),rank
849,849,849,NaN,Lewiston Elementary,K-8,1,0,0,0,Lewiston,Trinity,NaN,NaN,0
850,850,850,NaN,Taft City,K-8,5,1,0,0,Taft,Kern,NaN,NaN,0
851,851,851,NaN,Napa County Office Of Education,K-12,2,0,2,0,Napa,Napa,NaN,NaN,0
852,852,852,NaN,Mariposa County Office Of Education,K-12,1,0,0,2,Mariposa,Mariposa,NaN,NaN,0
853,853,853,NaN,Bridgeville Elementary,K-8,1,0,0,0,Bridgeville,Humboldt,NaN,NaN,0
854,854,854,NaN,Bangor Union Elementary,K-8,1,0,0,0,Bangor,Butte,NaN,NaN,0
855,855,855,NaN,Linns Valley-Poso Flat Union,K-8,1,0,0,0,Glennville,Kern,NaN,NaN,0
856,856,856,NaN,Baldy View Regional Occupational Program,9-12,0,0,1,0,Ontario,San Bernardino,NaN,NaN,0
857,857,857,NaN,Santa Clara County Regional Occupational Program,10-12,0,0,1,0,Sunnyvale,Santa Clara,NaN,NaN,0
858,858,858,NaN,Glenn County Regional Occupational Program,9-12,0,0,1,0,Willows,Glenn,NaN,NaN,0


In [19]:
len(schools[schools["Rank score (2017)"].isnull()])

245

In [20]:
len(schools)

1094

In [14]:
schools[schools["County"] == 'Amador']

,Unnamed: 0,index,Rank,District,Grades,# Elementary Schools,# Middle Schools,# High Schools,# Alt. Schools,City,County,Rank score (2017),Rank (2016),rank
419,419,419,422nd,Amador County Unified,K-12,6,2,3,1,Jackson,Amador,0.51,505.0,422
770,770,770,774th,Amador County Office Of Education,K-12,0,1,0,2,Jackson,Amador,0.22,NaN,774
978,978,978,NaN,Amador County Regional Occupational Program,12,0,0,1,0,Jackson,Amador,NaN,NaN,0


In [15]:
schools.head()

,Unnamed: 0,index,Rank,District,Grades,# Elementary Schools,# Middle Schools,# High Schools,# Alt. Schools,City,County,Rank score (2017),Rank (2016),rank
0,0,0,1st,Orinda Union Elementary,K-8,4,1,0,0,Orinda,Contra Costa,0.989,3.0,1
1,1,1,2nd,San Marino Unified,K-12,2,1,1,0,San Marino,Los Angeles,0.989,6.0,2
2,2,2,3rd,Rancho Santa Fe Elementary,K-8,1,1,0,0,Rancho Santa Fe,San Diego,0.985,10.0,3
3,3,3,4th,Montecito Union Elementary,K-6,1,0,0,0,Santa Barbara,Santa Barbara,0.985,2.0,4
4,4,4,5th,La Canada Unified,K-12,3,0,1,0,La Canada,Los Angeles,0.983,4.0,5


In [21]:
schools_by_county[schools_by_county["Rank score (2017)"].isnull()]

NameError: name 'schools_by_county' is not defined

In [22]:
schools_by_county = schools[['County', 'Rank score (2017)']].groupby("County").aggregate(np.nanmean).reset_index()
schools_by_county.head()

/Users/kanamishra/anaconda3/lib/python3.5/site-packages/pandas/core/groupby.py:842: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


,County,Rank score (2017)
0,Alameda,0.625222
1,Alpine,0.103000
2,Amador,0.365000
3,Butte,0.456600
4,Calaveras,0.510000


In [23]:
schools_by_county

,County,Rank score (2017)
0,Alameda,0.625222
1,Alpine,0.103000
2,Amador,0.365000
3,Butte,0.456600
4,Calaveras,0.510000
5,Colusa,0.484750
6,Contra Costa,0.632706
7,Del Norte,0.301000
8,El Dorado,0.624643
9,Fresno,0.357968


In [24]:
len(schools_by_county)

58

In [25]:
tracts_school = pd.merge(tracts_with_county, schools_by_county, how='left', left_on='NAME', right_on='County')

In [26]:
tracts_school.head()

,COUNTYFP,GEOID_x,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract,blkgrps,population,...,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu,COUNTYNS,GEOID_y,NAME,NAMELSAD,County,Rank score (2017)
0,001,06001442700,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3...",06001442700,3.0,3275.0,...,2581.0,1294.0,75.0,25.0,01675839,06001,Alameda,Alameda County,Alameda,0.625222
1,001,06001442800,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3...",06001442800,1.0,3056.0,...,2303.0,1958.0,71.0,29.0,01675839,06001,Alameda,Alameda County,Alameda,0.625222
2,037,06037204920,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3...",06037204920,2.0,2570.0,...,1605.0,1067.0,48.0,52.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235
3,037,06037205110,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3...",06037205110,1.0,3826.0,...,NaN,1140.0,0.0,100.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235
4,037,06037205120,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3...",06037205120,2.0,3288.0,...,NaN,766.0,12.0,88.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235


In [27]:
tracts_school["GEOID_x"].nunique()

8057

In [28]:
len(tracts_school)

8057

In [29]:
tracts_school.columns

Index(['COUNTYFP', 'GEOID_x', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry', 'tract', 'blkgrps', 'population', 'households',
       'land_acres', 'ht_ami', 'ht_80ami', 'ht_nmi', 'h_ami', 'h_80ami',
       'h_nmi', 't_ami', 't_80ami', 't_nmi', 'co2_per_hh_local',
       'co2_per_acre_local', 'autos_per_hh_ami', 'autos_per_hh_80ami',
       'autos_per_hh_nmi', 'vmt_per_hh_ami', 'vmt_per_hh_80ami',
       'vmt_per_hh_nmi', 'pct_transit_commuters_ami',
       'pct_transit_commuters_80ami', 'pct_transit_commuters_nmi',
       't_cost_ami', 't_cost_80ami', 't_cost_nmi', 'auto_ownership_cost_ami',
       'auto_ownership_cost_80ami', 'auto_ownership_cost_nmi', 'vmt_cost_ami',
       'vmt_cost_80ami', 'vmt_cost_nmi', 'transit_cost_ami',
       'transit_cost_80ami', 'transit_cost_nmi', 'transit_trips_ami',
       'transit_trips_80ami', 'transit_trips_nmi', 'compact_ndx',
       'emp_ovrll_ndx', 'res_density', 'gross_hh_density', 'hh_gravity',
       'frac_sfd', 'emp_gravity', 'emp

In [30]:
crimes.head()

,Unnamed: 0,County,Violent_sum,Property_sum
0,0,Alameda,10127,59744
1,1,Alpine,6,34
2,2,Amador,92,669
3,3,Butte,790,7307
4,4,Calaveras,178,656


In [31]:
tracts_final = pd.merge(tracts_school, crimes, how='left', on = 'County')
tracts_final.head()

,COUNTYFP,GEOID_x,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract,blkgrps,population,...,pct_renter_occupied_hu,COUNTYNS,GEOID_y,NAME,NAMELSAD,County,Rank score (2017),Unnamed: 0,Violent_sum,Property_sum
0,001,06001442700,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3...",06001442700,3.0,3275.0,...,25.0,01675839,06001,Alameda,Alameda County,Alameda,0.625222,0,10127,59744
1,001,06001442800,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3...",06001442800,1.0,3056.0,...,29.0,01675839,06001,Alameda,Alameda County,Alameda,0.625222,0,10127,59744
2,037,06037204920,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3...",06037204920,2.0,2570.0,...,52.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224
3,037,06037205110,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3...",06037205110,1.0,3826.0,...,100.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224
4,037,06037205120,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3...",06037205120,2.0,3288.0,...,88.0,00277283,06037,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224


In [32]:
len(tracts_final)

8057

In [33]:
tracts_final.drop("GEOID_y", axis = 1, inplace=True)

In [34]:
tracts_final.rename(columns={"GEOID_x" : "GEOID"}, inplace=True)

In [35]:
tracts_final.head()

,COUNTYFP,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract,blkgrps,population,...,pct_owner_occupied_hu,pct_renter_occupied_hu,COUNTYNS,NAME,NAMELSAD,County,Rank score (2017),Unnamed: 0,Violent_sum,Property_sum
0,001,06001442700,1234034,0,+37.5371514,-122.0081094,"POLYGON ((-122.017211 37.539323, -122.017186 3...",06001442700,3.0,3275.0,...,75.0,25.0,01675839,Alameda,Alameda County,Alameda,0.625222,0,10127,59744
1,001,06001442800,1278647,0,+37.5293619,-121.9931002,"POLYGON ((-122.002301 37.529836, -122.002236 3...",06001442800,1.0,3056.0,...,71.0,29.0,01675839,Alameda,Alameda County,Alameda,0.625222,0,10127,59744
2,037,06037204920,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.202837 34.019657, -118.202829 3...",06037204920,2.0,2570.0,...,48.0,52.0,00277283,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224
3,037,06037205110,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.219635 34.026285, -118.219449 3...",06037205110,1.0,3826.0,...,0.0,100.0,00277283,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224
4,037,06037205120,1466129,0,+34.0187546,-118.2117956,"POLYGON ((-118.220227 34.020564, -118.220183 3...",06037205120,2.0,3288.0,...,12.0,88.0,00277283,Los Angeles,Los Angeles County,Los Angeles,0.582235,18,56351,252224


In [36]:
tracts_final.columns

Index(['COUNTYFP', 'GEOID', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry', 'tract', 'blkgrps', 'population', 'households',
       'land_acres', 'ht_ami', 'ht_80ami', 'ht_nmi', 'h_ami', 'h_80ami',
       'h_nmi', 't_ami', 't_80ami', 't_nmi', 'co2_per_hh_local',
       'co2_per_acre_local', 'autos_per_hh_ami', 'autos_per_hh_80ami',
       'autos_per_hh_nmi', 'vmt_per_hh_ami', 'vmt_per_hh_80ami',
       'vmt_per_hh_nmi', 'pct_transit_commuters_ami',
       'pct_transit_commuters_80ami', 'pct_transit_commuters_nmi',
       't_cost_ami', 't_cost_80ami', 't_cost_nmi', 'auto_ownership_cost_ami',
       'auto_ownership_cost_80ami', 'auto_ownership_cost_nmi', 'vmt_cost_ami',
       'vmt_cost_80ami', 'vmt_cost_nmi', 'transit_cost_ami',
       'transit_cost_80ami', 'transit_cost_nmi', 'transit_trips_ami',
       'transit_trips_80ami', 'transit_trips_nmi', 'compact_ndx',
       'emp_ovrll_ndx', 'res_density', 'gross_hh_density', 'hh_gravity',
       'frac_sfd', 'emp_gravity', 'emp_n

In [66]:
tracts_final.to_csv("tracts_updated.csv")

In [1]:
print("test")

test
